<a href="https://colab.research.google.com/github/shivansh2310/Finance-Stuff/blob/main/CRT1_RM_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## delta co_var

In [1]:
import numpy as np
from statsmodels.regression.quantile_regression import QuantReg

def delta_co_var(output_returns, input_returns, lagged_factors_returns=None, quantile=0.05):
    num_periods = len(output_returns)
    median_percentile = 0.5

    # Preparing X matrix with intercept and lagged factors
    if lagged_factors_returns is None:
        X = np.ones((num_periods, 1))
    else:
        X = np.column_stack([np.ones(num_periods),
                            lagged_factors_returns[:-1]])

    y = input_returns

    # Calculating median state of input institution
    qr = QuantReg(y, X).fit(median_percentile)
    betas = qr.params
    if lagged_factors_returns is None:
        median_input_state = betas[0]
    else:
        median_input_state = np.array([1] + [lagged_factors_returns[-1]]) @ betas

    # Calculating distressed state of input institution
    qr = QuantReg(y, X).fit(quantile)
    betas = qr.params
    if lagged_factors_returns is None:
        distressed_input_state = betas[0]
    else:
        distressed_input_state = np.array([1] + [lagged_factors_returns[-1]]) @ betas

    # Quantile regression of output institution/system
    if lagged_factors_returns is None:
        X = np.column_stack([np.ones(num_periods), input_returns])
    else:
        X = np.column_stack([np.ones(num_periods),
                            input_returns,
                            lagged_factors_returns[:-1]])

    y = output_returns
    qr = QuantReg(y, X).fit(quantile)
    betas = qr.params

    # Calculating Delta CoVaR
    dcovar = betas[1] * (distressed_input_state - median_input_state)

    return dcovar

In [2]:
# Example
output_returns = np.array([-0.01, 0.03, -0.02, 0.05, 0.03, 0.03])
input_returns = np.array([0.01, 0.04, -0.02, -0.03, 0.004, 0.03])
lagged_factors_returns = np.array([0.07, 0.11, 0.10, 0.16, 0.08, 0.12, 0.01])

dcovar = delta_co_var(output_returns, input_returns, lagged_factors_returns, 0.05)
print("Delta CoVaR:", dcovar)

Delta CoVaR: -0.02976017146285906


## Co_risk

In [3]:
import numpy as np
from statsmodels.regression.quantile_regression import QuantReg

def co_risk(output_cds_spreads, input_cds_spreads, risk_factors_series, q, risk_factors_values, use_cvx=False):

    num_dates = len(output_cds_spreads)

    # Preparing design matrix with intercept, risk factors, and input CDS
    X = np.column_stack([
        np.ones(num_dates),
        risk_factors_series,
        input_cds_spreads
    ])

    y = output_cds_spreads

    # Quantile regression
    qr = QuantReg(y, X).fit(q=q)
    betas = qr.params

    # Calculating quantiles
    input_cds_quantile = np.percentile(input_cds_spreads, q * 100)
    output_cds_quantile = np.percentile(output_cds_spreads, q * 100)

    # Calculating co-risk measure
    numerator = (betas[0] +
                betas[-1] * output_cds_quantile +
                np.dot(betas[1:-1], risk_factors_values))

    c = 100 * ((numerator / input_cds_quantile) - 1)

    return c

# Example
output_cds_spreads = np.array([0.04, 0.01, 0.08, 0.09, 0.01])
input_cds_spreads = np.array([0.01, 0.01, 0.04, 0.03, 0.06])
risk_factors_series = np.array([
    [-1.2, 0.1],
    [-1.1, 0.7],
    [-0.8, 0.2],
    [-1.3, 1.8],
    [0.3, 2.3]
])
q = 0.95
risk_factors_values = np.array([-1.2, 0.1])

c = co_risk(output_cds_spreads, input_cds_spreads,
            risk_factors_series, q, risk_factors_values)
print(f"Co-Risk Measure: {c:.2f}%")

Co-Risk Measure: 341.68%


## Marginal & Systemic Shortfall

In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression

def marginal_expected_shortfall(firm_returns, market_returns):

    if len(firm_returns) != len(market_returns):
        raise ValueError("Unequal number of days for firm and market returns")

    # Finding the 5% quantile of market returns
    low_threshold = np.percentile(market_returns, 5)

    # Identifying worst days (market returns below 5% quantile)
    worst_days = market_returns < low_threshold

    # Calculating mean firm returns during worst market days
    mes = np.mean(firm_returns[worst_days])

    return mes


# Example
firm_returns = np.array([
    -0.1595, 0.1211, -0.0806, -0.0291, 0.0897,
    -0.0254, 0.1210, 0.0132, -0.1214, 0.1901, 0.0243
])
market_returns = np.array([
    -0.0205, -0.0510, 0.0438, 0.0914, 0.0243,
    -0.1051, 0.0121, 0.0221, -0.0401, -0.0111, -0.0253
])

mes = marginal_expected_shortfall(firm_returns, market_returns)
print(f"Marginal Expected Shortfall: {mes:.4f}")



Marginal Expected Shortfall: -0.0254


In [5]:
def systemic_expected_shortfall(mes_training, lvg_training, ses_training, mes_firm, lvg_firm):

    # Preparing design matrix
    X = np.column_stack([np.ones(len(mes_training)), mes_training, lvg_training])
    y = ses_training

    # Performing linear regression
    model = LinearRegression(fit_intercept=False).fit(X, y)
    betas = model.coef_

    # Calculating SES for the firm
    b, c = betas[1], betas[2]
    ses = (b * mes_firm + c * lvg_firm) / (b + c)

    return ses

    # Example
mes_training = np.array([-0.023, -0.07, 0.01])
lvg_training = np.array([1.8, 1.5, 2.2])
ses_training = np.array([0.3, 0.4, -0.2])
mes_firm = 0.04
lvg_firm = 1.7

ses = systemic_expected_shortfall(mes_training, lvg_training, ses_training, mes_firm, lvg_firm)
print(f"Systemic Expected Shortfall: {ses:.4f}")


Systemic Expected Shortfall: -0.3334
